# Methods of accessing data and metadata

## Intake

> "Taking the pain out of data access and distribution"

[Intake](https://intake.readthedocs.io/en/latest/) is a python package used to find, investigate, load, and disseminate data. 


#### Data loading
`Intake` can be used to load many different types of data formats (e.g. tabular data, multi-dimensional data, etc.) into a python notebook or script using familiar containers (e.g. Pandas dataframes, Xarray DataArrays, etc.). Intake can work on local, remote, or cloud computing infrastructures, and is relatively fast due to its ability to integrate distirbuted computing (e.g. Dask).

There are a number of [plugins](https://intake.readthedocs.io/en/latest/plugin-directory.html#plugin-directory) that currently exist for different types of data. Several that may be of particular interest to the climate science community are:

- [intake-esm](https://intake-esm.readthedocs.io/en/latest/): for ESM catalog datasets (including CMIP and CESM datasets)
- [intake-geopandas](https://github.com/intake/intake_geopandas): for reading in geospatial datasets into a [geopandas](https://geopandas.org) dataframe
- [intake-stac](https://intake-stac.readthedocs.io/en/latest/): for SpatioTemporal Asset Catalogs ([STAC](https://stacspec.org)); the [Pangeo Data Catalog](https://pangeo.io/catalog.html) uses STAC to catalog its cloud-based datasets
- [intake-thredds](https://intake-thredds.readthedocs.io/en/latest/): to retrieve data from THREDDS data servers (used by NCAR for example)
- [intake-xarray](https://intake-xarray.readthedocs.io/en/latest/): to load datasets into [Xarray](http://xarray.pydata.org/en/stable/) containers

#### Other features

`Intake` (and many of the plugins) can also be used for a few other tasks:
- cataloging system for listing data sources, metadata, and parameters
- convenience functions that can be used to, among other things, distribute data catalogs 
- investigate data sources and create plots using a GUI